https://github.com/Infatoshi/fcc-intro-to-llms/blob/main/bigram.ipynb

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device  = "cuda" if torch.cuda.is_available() else "cpu"


/home/raghu/anaconda3/envs/py/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392026823/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


### Data Loader

In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text), len(list(text)))
print(text[:100])


232310 232310
﻿  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(list(set(text)))
print(len(chars))
print(chars)


81
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {c:i for i,c in enumerate(chars)}
int_to_string = {i:c for i,c in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)


### Dataset Splits

In [5]:
train_len = int(len(data) * 0.8)
train_data = data[:train_len]
val_data = data[train_len:]

print(len(train_data), len(val_data))


185848 46462


### Hyperparameters

In [6]:
block_size = 64     # Try 32,  8
batch_size = 128    # 4
vocab_size = len(chars)
n_embd = 384
n_layer = 8          # 4
n_head = 8           # 4
dropout = 0.2

### Batch Function

In [7]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(f"Batch indices: {ix}")
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)


inputs:
tensor([[58,  9,  1,  ..., 67, 73,  1],
        [69, 62, 60,  ..., 78, 58, 57],
        [54,  2,  3,  ...,  5, 73,  1],
        ...,
        [ 1, 54, 72,  ..., 58, 60, 58],
        [68, 67,  1,  ...,  3,  0,  0],
        [72,  1, 60,  ..., 56, 62, 68]])
targets:
tensor([[ 9,  1, 54,  ..., 73,  1, 73],
        [62, 60, 65,  ..., 58, 57,  1],
        [ 2,  3,  1,  ..., 73,  1, 33],
        ...,
        [54, 72,  1,  ..., 60, 58, 73],
        [67,  1, 73,  ...,  0,  0, 51],
        [ 1, 60, 71,  ..., 62, 68, 74]])


In [8]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        #print(f"X Shape: {x.shape}")
        k = self.key(x)   # (B,T,hs)
        #print(f"K Shape: {k.shape}")
        q = self.query(x) # (B,T,hs)
        #print(f"Q Shape: {q.shape}")
        # compute attention scores ("affinities")
        # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 
        # (B, T, T)
        # tril converts upper half of the matrix to 0
        # masked_fill changes all these 0s to '-inf'
        #print(f"wei shape: {wei.shape}")
        #print(f"tril shape: {self.tril[:T, :T].shape}")
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        # (B, T, T)
        # softmax converts '-inf' to 0
        wei = F.softmax(wei, dim=-1) 
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        # (B,T,hs)
        v = self.value(x) 
        # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        out = wei @ v 
        return out


In [9]:
class MultiheadAttention(nn.Module):
    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(head_size * n_head, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3, h4, h4, h4, h4])
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [10]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)
    

In [11]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiheadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)     # Add and Norm
        y = self.ffwd(x)
        x = self.ln2(x + y)

        return x


In [17]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        
        # * means repeating Block for n_layer number of times
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def forward(self, index, targets=None):
        # index - context or sequence
        batch, seq_len = index.shape
        
        # (b, seq, vsize)
        tok_embd = self.token_embedding_table(index)
        # (seq, vsize)
        pos_embd = self.position_embedding_table(torch.arange(seq_len, device=device))
        # (b, seq, vsize)
        x = tok_embd + pos_embd

        x = self.blocks(x)
        x = self.ln_f(x)
        # (b, seq, vsize)
        logits = self.lm_head(x)


        if targets is None:
            loss = None
        else:
            # print(logits.shape)           # (4, 8, 81)
            # B(batch), T(time), C(channel)
            batch, seq_len, vocab_size = logits.shape
            # N -batch_size, C - number of classes as in pytorch docs
            logits = logits.view(batch * seq_len, vocab_size)
            targets = targets.view(batch * seq_len)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):

            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]

            # get the predictions
            logits, loss = self.forward(index_cond)

            # Focus only on the last time step
            logits = logits[:, -1, :]   # (B, C)

            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)   # (B, C)

            # Sample from the distribution
            ind1 = torch.multinomial(probs, num_samples=1)

            index = torch.cat((index, ind1), dim=1) # (B, T+1)

        return index
  

In [13]:
model = GPTLanguageModel(vocab_size).to(device)
m = model.to(device)


In [14]:
max_iters = 200
# eval_interval = 2500
learning_rate = 3e-4    # Other options: 1e-3, 3e-3, 1e-4
eval_iters = 100

# Create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

@torch.no_grad()
def estimate_loss():
    out = {}
    losses = torch.zeros(eval_iters)

    model.eval()    
    for iter in range(eval_iters):
        X, Y = get_batch('validation')
        logits, loss = model(X, Y)
        losses[iter] = loss.item()

    return losses.mean()


In [15]:
# Training loop
for iter in range(max_iters):

    # sample a batch of  data
    xb, yb = get_batch('train')

    model.train()

    # Forward pass and evaluate the loss
    logits, loss = model(xb, yb)    # OR model.forward(xb, yb)

    # Step 3 - Computes gradients for both "b" and "w" parameters
    loss.backward()

    # Step 4 - Updates parameters using gradients and the learning rate
    optimizer.step()

    # Step 5 - After applying gradients, reset gradients for the next loop
    optimizer.zero_grad(set_to_none=True)

    if iter % eval_iters == 0:
        val_loss = estimate_loss()
        print(f"Step: {iter}, Training Loss: {loss:.3f}, Validation Loss: {val_loss:.3f}")

print(loss.item())

# -ln(0.14) = 1.96
# Model has about a 14 percent chance of guessing the next token

Step: 0, Training Loss: 4.384, Validation Loss: 3.530
Step: 100, Training Loss: 2.306, Validation Loss: 2.360
1.926596999168396


In [18]:
context = torch.zeros((1,1), dtype=torch.long, device=device)   # tensor([[0.]])
output = m.generate(context, max_new_tokens=500)
output.shape


IndexError: index out of range in self

In [ ]:
output = output[0].tolist()
generated_chars = decode(output)
print(generated_chars)


In [ ]:
next(model.parameters()).is_cuda


False

In [ ]:
torch.cuda.is_available()


False